# AI Mid Lab
# Name : Muhammad Ismail
# Reg no. Sp23-Bse-145
# Topic: Missionaries and Cannibal Problem

In [ ]:
import collections
import heapq

# --- Configuration ---
MAX_PEOPLE = 3  # Number of missionaries and cannibals
START_STATE = (MAX_PEOPLE, MAX_PEOPLE, 'L')  # (M_left, C_left, Boat_pos)
GOAL_STATE = (0, 0, 'R')

# Possible moves in terms of (Missionaries, Cannibals)
# The boat can carry (1M, 0C), (2M, 0C), (0M, 1C), (0M, 2C), or (1M, 1C)
MOVES = [(1, 0), (2, 0), (0, 1), (0, 2), (1, 1)]

# --- Helper Functions ---

def is_valid_state(m_left, c_left):
    """
    Checks if a given state (M_left, C_left) is safe based on the rules.
    The rules apply to both banks.
    """
    if not (0 <= m_left <= MAX_PEOPLE and 0 <= c_left <= MAX_PEOPLE):
        return False

    # Check Left Bank
    # If there are missionaries on the left, cannibals cannot outnumber them.
    if m_left > 0 and c_left > m_left:
        return False

    # Calculate people on the Right Bank
    m_right = MAX_PEOPLE - m_left
    c_right = MAX_PEOPLE - c_left

    # Check Right Bank
    # If there are missionaries on the right, cannibals cannot outnumber them.
    if m_right > 0 and c_right > m_right:
        return False

    return True

def get_successor_states(current_state):
    """
    Generates all valid successor states from the current state.
    """
    m_left, c_left, boat_pos = current_state
    successors = []

    for dm, dc in MOVES:
        if boat_pos == 'L':
            # Boat moves Left to Right (L -> R)
            new_m_left = m_left - dm
            new_c_left = c_left - dc
            new_boat_pos = 'R'

            # Check if there are enough people on the left bank for the move
            if m_left >= dm and c_left >= dc:
                if is_valid_state(new_m_left, new_c_left):
                    successors.append((new_m_left, new_c_left, new_boat_pos))

        elif boat_pos == 'R':
            # Boat moves Right to Left (R -> L)
            # Calculate people on the right bank to check if they can make the move
            m_right = MAX_PEOPLE - m_left
            c_right = MAX_PEOPLE - c_left

            new_m_left = m_left + dm
            new_c_left = c_left + dc
            new_boat_pos = 'L'

            # Check if there are enough people on the right bank for the move
            if m_right >= dm and c_right >= dc:
                if is_valid_state(new_m_left, new_c_left):
                    successors.append((new_m_left, new_c_left, new_boat_pos))

    return successors

def reconstruct_path(parent_map, goal_state):
    """Reconstructs the path from start to goal using the parent map."""
    path = []
    current = goal_state
    while current is not None:
        path.append(current)
        current = parent_map.get(current)
    path.reverse()
    return path

def display_path(path, algorithm_name):
    """Prints the found path in a readable format."""
    print(f"\n--- Solution found using {algorithm_name} ---")
    if not path:
        print("No solution found.")
        return

    m_start, c_start, _ = START_STATE
    print(f"Start State: ({m_start} Missionaries, {c_start} Cannibals) on Left Bank | Boat on Left")

    for i, state in enumerate(path):
        m_left, c_left, boat_pos = state
        m_right = MAX_PEOPLE - m_left
        c_right = MAX_PEOPLE - c_left
        bank_info = f"({m_left} M, {c_left} C) left | ({m_right} M, {c_right} C) right"

        step_description = f"Step {i}: "
        if i == 0:
            step_description += "Initial State"
        elif i == len(path) - 1:
            step_description += "Goal Reached"
        else:
            # Determine the move made between path[i-1] and path[i]
            prev_m, prev_c, prev_boat = path[i-1]
            dm = abs(m_left - prev_m)
            dc = abs(c_left - prev_c)
            direction = "L -> R" if boat_pos == 'R' else "R -> L"

            step_description += f"Move {dm} M, {dc} C {direction}"

        print(f"  {step_description:25} | State: {state} | Banks: {bank_info}")

# --- Search Algorithms ---

def breadth_first_search():
    """Finds the shortest path using BFS."""
    # Queue for the frontier: stores states to visit
    queue = collections.deque([START_STATE])

    # Set to track visited states
    visited = {START_STATE}

    # Dictionary to reconstruct the path: child_state -> parent_state
    parent_map = {START_STATE: None}

    while queue:
        current_state = queue.popleft()

        if current_state == GOAL_STATE:
            return reconstruct_path(parent_map, current_state)

        for next_state in get_successor_states(current_state):
            if next_state not in visited:
                visited.add(next_state)
                parent_map[next_state] = current_state
                queue.append(next_state)

    return None

def depth_first_search():
    """Finds a path using DFS (Stack-based implementation)."""
    # Stack for the frontier: stores states to visit (LIFO)
    stack = [START_STATE]

    # Set to track visited states (to prevent infinite loops in cycles)
    visited = {START_STATE}

    # Dictionary to reconstruct the path
    parent_map = {START_STATE: None}

    while stack:
        current_state = stack.pop()

        if current_state == GOAL_STATE:
            return reconstruct_path(parent_map, current_state)

        # Iterate over successors in reverse order if you want to match the BFS order
        # but standard DFS order is usually arbitrary, just ensuring we explore.
        for next_state in get_successor_states(current_state):
            if next_state not in visited:
                visited.add(next_state)
                parent_map[next_state] = current_state
                stack.append(next_state)

    return None

def a_star_search():
    """
    Finds the shortest path using A* Search.
    Heuristic h(n): Estimated number of trips remaining.
    It's the remaining people on the left bank divided by the max boat capacity (2), rounded up.
    This is admissible (never overestimates) as at most 2 people can cross per trip.
    """
    def heuristic(state):
        m_left, c_left, boat_pos = state
        return (m_left + c_left + 1) // 2 # ceil((M+C)/2)

    # Priority Queue for the frontier: stores (f_cost, g_cost, state)
    # The smallest f_cost (g + h) is always popped first.
    g_cost = 0
    h_cost = heuristic(START_STATE)
    f_cost = g_cost + h_cost

    # Priority Queue structure: (f_cost, g_cost, state)
    # We use g_cost as the secondary sort key to break ties (lower g_cost first)
    # The state must be the last element for easy access.
    frontier = [(f_cost, g_cost, START_STATE)]

    # Dictionary to store the lowest g_cost found for each state
    g_costs = {START_STATE: 0}

    # Dictionary to reconstruct the path
    parent_map = {START_STATE: None}

    while frontier:
        # Pop the state with the lowest f_cost
        f_current, g_current, current_state = heapq.heappop(frontier)

        if current_state == GOAL_STATE:
            return reconstruct_path(parent_map, current_state)

        for next_state in get_successor_states(current_state):
            # The cost to move from current to next is always 1 (one trip)
            g_new = g_current + 1

            # If we found a shorter path to next_state, update its record
            if next_state not in g_costs or g_new < g_costs[next_state]:
                g_costs[next_state] = g_new
                parent_map[next_state] = current_state

                h_new = heuristic(next_state)
                f_new = g_new + h_new

                # Push the updated state information to the frontier
                heapq.heappush(frontier, (f_new, g_new, next_state))

    return None

# --- Colab Execution Functions ---

def run_bfs():
    """Runs and displays the Breadth-First Search solution."""
    print("Missionaries and Cannibals River Crossing Puzzle (3 M, 3 C)")
    bfs_path = breadth_first_search()
    display_path(bfs_path, "Breadth-First Search (BFS)")

def run_dfs():
    """Runs and displays the Depth-First Search solution."""
    print("Missionaries and Cannibals River Crossing Puzzle (3 M, 3 C)")
    dfs_path = depth_first_search()
    display_path(dfs_path, "Depth-First Search (DFS)")

def run_a_star():
    """Runs and displays the A* Search Algorithm solution."""
    print("Missionaries and Cannibals River Crossing Puzzle (3 M, 3 C)")
    a_star_path = a_star_search()
    display_path(a_star_path, "A* Search Algorithm")

# --- Run the search algorithms ---
run_bfs()
run_dfs()
run_a_star()

Missionaries and Cannibals River Crossing Puzzle (3 M, 3 C)

--- Solution found using Breadth-First Search (BFS) ---
Start State: (3 Missionaries, 3 Cannibals) on Left Bank | Boat on Left
  Step 0: Initial State     | State: (3, 3, 'L') | Banks: (3 M, 3 C) left | (0 M, 0 C) right
  Step 1: Move 0 M, 2 C L -> R | State: (3, 1, 'R') | Banks: (3 M, 1 C) left | (0 M, 2 C) right
  Step 2: Move 0 M, 1 C R -> L | State: (3, 2, 'L') | Banks: (3 M, 2 C) left | (0 M, 1 C) right
  Step 3: Move 0 M, 2 C L -> R | State: (3, 0, 'R') | Banks: (3 M, 0 C) left | (0 M, 3 C) right
  Step 4: Move 0 M, 1 C R -> L | State: (3, 1, 'L') | Banks: (3 M, 1 C) left | (0 M, 2 C) right
  Step 5: Move 2 M, 0 C L -> R | State: (1, 1, 'R') | Banks: (1 M, 1 C) left | (2 M, 2 C) right
  Step 6: Move 1 M, 1 C R -> L | State: (2, 2, 'L') | Banks: (2 M, 2 C) left | (1 M, 1 C) right
  Step 7: Move 2 M, 0 C L -> R | State: (0, 2, 'R') | Banks: (0 M, 2 C) left | (3 M, 1 C) right
  Step 8: Move 0 M, 1 C R -> L | State: (0, 3, 